In [2]:
import numpy as np
from sklearn.datasets import make_classification
class LayerDense:
    def __init__(self, n_features, n_neurons):
        self.weights = 0.1 * np.random.randn(n_neurons, n_features)
        self.bias = np.zeros((n_neurons,1))
        
    def forward(self, inputs):
        self.output = np.dot(self.weights, inputs) +self.bias

class Activation_ReLU:
    def forward(self, inputs):
        self.output= np.maximum(0, inputs)

class Activation_Softmax:
    def forward(self, inputs):
        exp_values= np.exp(inputs - np.max(inputs, axis=0, keepdims=True))
        self.output = exp_values / np.sum(exp_values, axis=0, keepdims=True)
class Activation_sigmoid:
    def forward(self, inputs):
        self.output= 1/(1+np.exp(-inputs))
    

In [3]:
import pandas as pd
data = pd.read_csv('MedicalAppointment.csv')

In [4]:
data['Age'].describe()

count    110527.000000
mean         37.088874
std          23.110205
min          -1.000000
25%          18.000000
50%          37.000000
75%          55.000000
max         115.000000
Name: Age, dtype: float64

In [ ]:
data = data.drop(index=99832)


In [ ]:
data[data['No-show']=='No']

In [ ]:
x = data[['Age', 'Scholarship', 'Hipertension', 'Diabetes', 'Alcoholism', 'Handcap', 'SMS_received']].to_numpy()

In [ ]:
y = data['No-show'].to_numpy()

In [ ]:
y= np.where(y=='No', 1,0)

In [ ]:
y.shape

In [ ]:
def cost(y_hat, y):
    y_hat = np.clip(y_hat, 1e-7, 1-1e-7)

    loss = -y*np.log(y_hat) -(1-y)*np.log(1-y_hat)
    if (loss.shape[1] !=1):
        summed =1/len(y)* np.sum(loss, axis=1, keepdims=1)
    else:
        summed =1/len(y)*loss
    return summed



In [ ]:
np.array([1,2])*np.array([2,4])

In [ ]:
m = len(data)

In [ ]:
x = x.reshape((7,110526))

In [ ]:
layer1 = LayerDense(7, 256)
layer2 = LayerDense(256,1)

activation1 = Activation_sigmoid()
activation2 = Activation_sigmoid()
y = y.reshape(1,110526)
lr = 0.03

In [ ]:
def backward_layer2(y_hat, y, m, a1, w2):
    a3 = y_hat
    dc_dz2 = (y_hat - y)/m
    dz2_dw2 =   a1
   
    dw = dc_dz2@dz2_dw2.T
    db = np.sum(dc_dz2)
    dz2_da1 = w2

    dc_da1 = w2.T@dc_dz2

    return dw, db, dc_da1

In [ ]:
m= len(data)

In [ ]:
def backward_layer1(dc_da1, a1, w1, x):
    dc_dz1= dc_da1 * a1*(1-a1)
    dz1_dw1 = x
    dc_dw1 = dc_dz1@dz1_dw1.T
    dc_db1 = np.sum(dc_dz1, axis=1, keepdims=1)

    return dc_dw1, dc_db1

In [ ]:
import tracemalloc


In [ ]:
tracemalloc.start()
for _ in range(120):
    layer1.forward(x)
    activation1.forward(layer1.output)
    layer2.forward(activation1.output)
    activation2.forward(layer2.output)
    loss = cost(activation2.output, y)
    dw2, db2, dc_da1 = backward_layer2(activation2.output, y, x.shape[1], activation1.output, layer2.weights)
    dw1,db1  = backward_layer1(dc_da1, activation1.output, layer1.weights, x)
    layer2.weights -= lr*dw2
    layer1.weights -= lr*dw1
    layer2.bias -= lr*db2
    layer1.bias -= lr*db1
print(tracemalloc.get_tracemalloc_memory())
tracemalloc.stop()

In [ ]:
y_pred = activation2.output

In [ ]:
y_pred = np.where(y_pred>0.6, 1,0)

In [ ]:
y_pred = np.where(y_pred>0.5,1, 0)

In [ ]:
y_pred

In [ ]:
y

In [ ]:
y_pred, y =y_pred.reshape((m, 1)), y.reshape((m,1))

In [ ]:
from sklearn.metrics import accuracy_score , f1_score, roc_auc_score, confusion_matrix
accuracy_score(y, y_pred)

In [ ]:
f1_score(y, y_pred)

In [ ]:
roc_auc_score(y, y_pred)

In [ ]:
confusion_matrix(y, y_pred)